In [ ]:
pip install xgboost lightgbm catboost

In [22]:
import pandas as pd
import numpy as np


from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from catboost import CatBoostClassifier
from sklearn.metrics import log_loss
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import StratifiedKFold

pd.set_option('display.max_columns', 500)

In [6]:
train = pd.read_csv('Data/train.csv')
test = pd.read_csv('Data/test.csv')
greeks = pd.read_csv('Data/greeks.csv')
sub = pd.read_csv('Data/sample_submission.csv')

In [9]:
LE = LabelEncoder()

train['EJ'] = LE.fit_transform(train['EJ'])
test['EJ'] = LE.transform(test['EJ'])

In [7]:
train.head()

,Id,AB,AF,AH,AM,AR,AX,AY,AZ,BC,BD,BN,BP,BQ,BR,BZ,CB,CC,CD,CF,CH,CL,CR,CS,CU,CW,DA,DE,DF,DH,DI,DL,DN,DU,DV,DY,EB,EE,EG,EH,EJ,EL,EP,EU,FC,FD,FE,FI,FL,FR,FS,GB,GE,GF,GH,GI,GL,Class
0,000ff2bfdfe9,0.209377,3109.03329,85.200147,22.394407,8.138688,0.699861,0.025578,9.812214,5.555634,4126.58731,22.5984,175.638726,152.707705,823.928241,257.432377,47.223358,0.563481,23.387600,4.851915,0.023482,1.050225,0.069225,13.784111,1.302012,36.205956,69.08340,295.570575,0.23868,0.284232,89.245560,84.31664,29.657104,5.310690,1.74307,23.187704,7.294176,1.987283,1433.166750,0.949104,B,30.879420,78.526968,3.828384,13.394640,10.265073,9028.291921,3.583450,7.298162,1.73855,0.094822,11.339138,72.611063,2003.810319,22.136229,69.834944,0.120343,1
1,007255e47698,0.145282,978.76416,85.200147,36.968889,8.138688,3.632190,0.025578,13.517790,1.229900,5496.92824,19.4205,155.868030,14.754720,51.216883,257.432377,30.284345,0.484710,50.628208,6.085041,0.031442,1.113875,1.117800,28.310953,1.357182,37.476568,70.79836,178.553100,0.23868,0.363489,110.581815,75.74548,37.532000,0.005518,1.74307,17.222328,4.926396,0.858603,1111.287150,0.003042,A,109.125159,95.415086,52.260480,17.175984,0.296850,6785.003474,10.358927,0.173229,0.49706,0.568932,9.292698,72.611063,27981.562750,29.135430,32.131996,21.978000,0
2,013f2bd269f5,0.470030,2635.10654,85.200147,32.360553,8.138688,6.732840,0.025578,12.824570,1.229900,5135.78024,26.4825,128.988531,219.320160,482.141594,257.432377,32.563713,0.495852,85.955376,5.376488,0.036218,1.050225,0.700350,39.364743,1.009611,21.459644,70.81970,321.426625,0.23868,0.210441,120.056438,65.46984,28.053464,1.289739,1.74307,36.861352,7.813674,8.146651,1494.076488,0.377208,B,109.125159,78.526968,5.390628,224.207424,8.745201,8338.906181,11.626917,7.709560,0.97556,1.198821,37.077772,88.609437,13676.957810,28.022851,35.192676,0.196941,0
3,043ac50845d5,0.252107,3819.65177,120.201618,77.112203,8.138688,3.685344,0.025578,11.053708,1.229900,4169.67738,23.6577,237.282264,11.050410,661.518640,257.432377,15.201914,0.717882,88.159360,2.347652,0.029054,1.400300,0.636075,41.116960,0.722727,21.530392,47.27586,196.607985,0.23868,0.292431,139.824570,71.57120,24.354856,2.655345,1.74307,52.003884,7.386060,3.813326,15691.552180,0.614484,B,31.674357,78.526968,31.323372,59.301984,7.884336,10965.766040,14.852022,6.122162,0.49706,0.284466,18.529584,82.416803,2094.262452,39.948656,90.493248,0.155829,0
4,044fb8a146ec,0.380297,3733.04844,85.200147,14.103738,8.138688,3.942255,0.054810,3.396778,102.151980,5728.73412,24.0108,324.546318,149.717165,6074.859475,257.432377,82.213495,0.536467,72.644264,30.537722,0.025472,1.050225,0.693150,31.724726,0.827550,34.415360,74.06532,200.178160,0.23868,0.207708,97.920120,52.83888,26.019912,1.144902,1.74307,9.064856,7.350720,3.490846,1403.656300,0.164268,B,109.125159,91.994825,51.141336,29.102640,4.274640,16198.049590,13.666727,8.153058,48.50134,0.121914,16.408728,146.109943,8524.370502,45.381316,36.262628,0.096614,1


In [19]:
test

,Id,AB,AF,AH,AM,AR,AX,AY,AZ,BC,BD,BN,BP,BQ,BR,BZ,CB,CC,CD,CF,CH,CL,CR,CS,CU,CW,DA,DE,DF,DH,DI,DL,DN,DU,DV,DY,EB,EE,EG,EH,EJ,EL,EP,EU,FC,FD,FE,FI,FL,FR,FS,GB,GE,GF,GH,GI,GL
0,00eed32682bb,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,010ebe33f668,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,02fa521e1838,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,040e15f562a2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,046e85c7cc7f,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [10]:
greeks.head()

,Id,Alpha,Beta,Gamma,Delta,Epsilon
0,000ff2bfdfe9,B,C,G,D,3/19/2019
1,007255e47698,A,C,M,B,Unknown
2,013f2bd269f5,A,C,M,B,Unknown
3,043ac50845d5,A,C,M,B,Unknown
4,044fb8a146ec,D,B,F,B,3/25/2020


In [12]:
print(train.shape)
print(test.shape)
print(greeks.shape)

(617, 58)
(5, 57)
(617, 6)


In [17]:
missing = train.isna().sum().reset_index()
missing.columns = ['columns', 'missing_count']
missing.sort_values('missing_count', ascending = False).head(10)

,columns,missing_count
41,EL,60
13,BQ,60
17,CC,3
50,FS,2
16,CB,2
48,FL,1
44,FC,1
33,DU,1
56,GL,1
37,EE,0


In [14]:
train['Id'].nunique()

617

### Modelling

In [ ]:
## Defining the input and target variables
X = train.drop(columns = ['Id', 'class'], axis = 1)
Y = train['Class']

## Defining lists to store results
lgb_cv_scores, lgb_preds = list(), list()
xgb_cv_scores, xgb_preds = list(), list()
cat_cv_scores, cat_preds = list(), list()
ens_cv_scores, ens_preds = list(), list()

## Performing KFold cross-validation
skf = StratifiedKFold(n_splits = 10, shuffle = True, random_state = 42)
    
for i, (train_idx, valid_idx) in enumerate(skf.split(X, Y)):
        
    X_train, X_valid = X.iloc[train_idx], X.iloc[valid_idx]
    Y_train, Y_valid = Y.iloc[train_idx], Y.iloc[valid_idx]
    
    print('---------------------------------------------------------------')    
    
    lgbm_md = LGBMClassifier(objective = 'logloss', class_weight = 'balanced', verbosity = -1).fit(X_train, Y_train)
    
    lgbm_pred_valid = lgbm_md.predict_proba(X_valid)
    lgbm_pred_test = lgbm_md.predict_proba(X_test)
    
    lgbm_score_fold = log_loss(Y_valid, lgbm_pred_valid)
    lgbm_cv_scores.append(lgbm_score_fold)
    lgbm_preds.append(lgbm_pred_test)
    
    print('Fold', i+1, '==> LightGBM oof log-loss is ==>', lgbm_score_fold)  
    
    
    xgb_md = XGBClassifier(objective = 'binary:logistic', scale_pos_weight = 4.71, verbosity = 0).fit(X_train, Y_train)
    
    xgb_pred_valid = xgb_md.predict_proba(X_valid)
    xgb_pred_test = xgb_md.predict_proba(X_test)
    
    xgb_score_fold = log_loss(Y_valid, xgb_pred_valid)
    xgb_cv_scores.append(xgb_score_fold)
    xgb_preds.append(xgb_pred_test)
    
    print('Fold', i+1, '==> XGBoost oof log-loss is ==>', xgb_score_fold)
    
    
    cat_md = CatBoostClassifier(auto_class_weights = 'Balanced').fit(X_train, Y_train)
    
    cat_pred_valid = cat_md.predict_proba(X_valid)
    cat_pred_test = cat_md.predict_proba(X_test)
    
    cat_score_fold = log_loss(Y_valid, cat_pred_valid)
    cat_cv_scores.append(cat_score_fold)
    cat_preds.append(cat_pred_test)
    
    print('Fold', i+1, '==> CatBoost oof log-loss is ==>', cat_score_fold)
    
    
    
    
    
    
    x = pd.DataFrame({'HIST': hist_pred_1, 'LGB': lgb_pred_1, 'XGB': xgb_pred_1})
    y = Y_test
    
    lad_md = LADRegression().fit(x, y)
    lad_pred = lad_md.predict(x); lad_pred_round = np.round_(lad_pred).astype(int)
    
    x_test = pd.DataFrame({'HIST': hist_pred_2, 'LGB': lgb_pred_2, 'XGB': xgb_pred_2})
    lad_pred_test = lad_md.predict(x_test); lad_pred_test_round = np.round_(lad_pred_test).astype(int)
        
    ens_score = mean_absolute_error(y, lad_pred); ens_score_round = mean_absolute_error(y, lad_pred_round)
    ens_cv_scores.append(ens_score); ens_cv_scores_round.append(ens_score_round)
    ens_preds.append(lad_pred_test); ens_preds_round.append(lad_pred_test_round)
    
    print('Fold', i+1, '==> LAD ensemble oof MAE is ==>', ens_score)
    print('Fold', i+1, '==> LAD ensemble oof MAE is ==>', ens_score_round)
    
    
    x = pd.DataFrame({'HIST': hist_pred_1_round, 'LGB': lgb_pred_1_round, 'XGB': xgb_pred_1_round})
    y = Y_test
    
    lad_md = LADRegression().fit(x, y)
    lad_pred = lad_md.predict(x); lad_pred_round = np.round_(lad_pred).astype(int)
    
    x_test = pd.DataFrame({'HIST': hist_pred_2_round, 'LGB': lgb_pred_2_round, 'XGB': xgb_pred_2_round})
    lad_pred_test = lad_md.predict(x_test); lad_pred_test_round = np.round_(lad_pred_test).astype(int)
    
    ens_score = mean_absolute_error(y, lad_pred); ens_score_round = mean_absolute_error(y, lad_pred_round)
    ens_cv_scores2.append(ens_score); ens_cv_scores_round2.append(ens_score_round)
    ens_preds2.append(lad_pred_test); ens_preds_round2.append(lad_pred_test_round)
    
    print('Fold', i+1, '==> LAD Rounded ensemble oof MAE is ==>', ens_score)
    print('Fold', i+1, '==> LAD Rounded ensemble oof MAE is ==>', ens_score_round)